In [ ]:
import os
current_dir = os.getcwd()
os.chdir(current_dir)
import matplotlib.pyplot as plt
#%matplotlib inline  
#%load_ext autoreload
#%autoreload 2
import numpy as np
import time
import random
import pandas as pd
from joblib import Parallel, delayed

In [ ]:
def print_np(x):
    print ("Type is %s" % (type(x)))
    print ("Shape is %s" % (x.shape,))
#     print ("Values are: \n%s" % (x))

In [ ]:
import sys
# sys.path.append('../')
#sys.path.append('/')
sys.path.append(os.path.join(current_dir, 'constraints'))
sys.path.append(os.path.join(current_dir, 'model'))
sys.path.append(os.path.join(current_dir, 'cost'))

import UnicycleModel

#%%
import UnicycleCost
import UnicycleConstraints
from scipy.integrate import solve_ivp
from PTR_tf_free_v2 import PTR_tf_free

In [ ]:
def run_scp(i):
    ix = 3
    iu = 2
    ih = 2
    N = 30
    tf = 3
    delT = tf/N
    max_iter = 20

    mu=0
    sigma=0.5

    xi = np.zeros(3)
    xi[0] = -1.0 +np.random.normal(mu,sigma)
    xi[1] = -2.0 +np.random.normal(mu,sigma)
    xi[2] = 0 

    xf = np.zeros(3)
    xf[0] = 2.0 +np.random.normal(mu,sigma)
    xf[1] = 2.0 +np.random.normal(mu,sigma)
    xf[2] = 0

    myModel = UnicycleModel.unicycle('Hello',ix,iu,'numeric_central')
    myCost = UnicycleCost.unicycle('Hello',ix,iu,N)
    myConst = UnicycleConstraints.UnicycleConstraints('Hello',ix,iu)

    x0 = np.zeros((N+1,ix))
    for j in range(N+1) :
        x0[j] = (N-j)/N * xi + j/N * xf
    # u0 = np.random.rand(N,iu)
    u0 = np.zeros((N+1,iu))

    i1 = PTR_tf_free('unicycle',N,tf,max_iter,myModel,myCost,myConst,type_discretization="zoh",
            w_c=1,w_vc=1e3,w_tr=1e-1,w_rate=0,
            tol_vc=1e-6,tol_tr=1e-3)
    x,u,xbar,ubar,tf,total_num_iter,flag_boundary,l,l_vc,l_tr,x_traj,u_traj,T_traj = i1.run(x0,u0,xi,xf)
    xini = np.vstack((xi,xf))

    return x0,x,u,total_num_iter,xini,tf

In [ ]:
N=30
M=20000

results = Parallel(n_jobs=11)(delayed(run_scp)(i) for i in range(M))

In [ ]:
Totalx0 = np.concatenate([r[0] for r in results], axis=1)  # shape: (N+1, 3*M)
Totalx  = np.concatenate([r[1] for r in results], axis=1)  # shape: (N+1, 3*M)

v_all = np.stack([r[2][:, 0] for r in results], axis=1)  # (N+1, M)
w_all = np.stack([r[2][:, 1] for r in results], axis=1)  # (N+1, M)
Totalu = np.empty((N+1, 2*M))
Totalu[:, 0::2] = v_all
Totalu[:, 1::2] = w_all
Totalv = v_all
Totalw = w_all

Totaliter = np.array([r[3] for r in results]).reshape(1, M)

Totalxini = np.stack([r[4] for r in results], axis=1)  # shape: (6, M)

Totaltf = np.array([r[5] for r in results]).reshape(1, M)

In [ ]:
np.savez('data_unicycle_20000_sig0.5_tf_free', Totalx0=Totalx0, Totalx=Totalx, Totalu=Totalu,Totalv=Totalv, Totalw=Totalw, Totaliter=Totaliter,Totalxini=Totalxini,Totaltf=Totaltf)